In [ ]:
%%init_spark
launcher.jars = ["/app/setup/commons.jar"]
launcher.conf.spark.app.name = "bhavesh_notebook"
launcher.conf.spark.queue = "default"
launcher.conf.spark.local.dir = "/app/tmp"
launcher.conf.spark.sql.shuffle.partitions = 210
launcher.conf.spark.sql.shuffle.minPartitions = 20
launcher.conf.spark.driver.memory = "50g"
launcher.conf.spark.ui.showConsoleProgress = "true"
launcher.master = "local[15]"

In [ ]:
import ai.couture.obelisk.commons.Constants.{DB_PRODUCT_INTERACTIONS_PATH, PROCESSED_GA_DATA, GA_USER_CLICK_DATA}
import ai.couture.obelisk.commons.Constants.STANDARD_COL_NAMES._
import ai.couture.obelisk.commons.Constants._
import ai.couture.obelisk.commons.io._
import ai.couture.obelisk.commons.utils.DateTimeUtil._
import org.apache.spark.sql._
import org.apache.spark.sql.expressions._
import org.apache.spark.sql.functions._
import org.apache.spark.sql.types._

import spark.implicits._

// val B2B_PLP_RELEVANCE_DAG_INIT_PATH: String = "/data/ecomm/ajiob2b/metadata/plp/init"
// val LIVE_STYLES_DATE_WISE_PATH = "/data/ecomm/ajiob2b/processed/LiveStyleCodesDateWise"
// val LIVE_STYLES_DATE_WISE_PATH_V2 = "/data/ecomm/ajiob2b/processed/LiveStyleCodesDateWiseV2"
// val COMBINED_USER_PRODUCT_INTERACTIONS_PATH = "/data/ecomm/ajiob2b/processed/CombinedUPI"
// val GA_INTERACTIONS_PATH = "/data/ecomm/ajiob2b/processed/ga/ULevelProductGA"
// val USER_CLUSTER_MAPPING_PATH = "/data/ecomm/ajiob2b/processed/output/clustering/AssignedUsersClusters.json"
// val WISHLIST_PATH = "/data/ecomm/ajiob2b/processed/WishlistMaster"
val DAYS: String = "daysInHis"
val DATASET: String = "dataset"
val MODEL: String = "model"
val TARGET: String = "target"
val REVENUE: String = "revenue"
val DISTINCT_PURCHASERS: String = "distinctPurchasers"
val SALES = "sales"
val REVENUE_PER_VIEW: String = "RevenuePerView"
val QUANTITY_PER_VIEW: String = "QuantityPerView"
val WISHLIST_PER_VIEW: String = "WishlistPerView"
val CONSIDERATION: String = "Consideration"
val CONVERSION: String = "Conversion"
val PRODUCT_CLICKS: String = "productClicks"
val PRODUCT_VIEWS: String = "productViews"
val L1_NAME: String = "L1_NAME"
val L2_NAME: String = "L2_NAME"

// preprod paths
val B2B_PLP_RELEVANCE_DAG_INIT_PATH: String = "/data/ecomm/ajiob2b/metadata/plp/init"
val B2B_PLP_UC_DAG_INIT_PATH: String = "/data/ecomm/ajiob2b/metadata/plp/uc/init"
val STYLECODE_TO_JIOCODE_MAPPING_PATH: String = "/data/ecomm/ajiob2b/processed/20230511/StyleCodeToJioCodeMapping"
val B2B_BRICK_DETAILS_PATH: String = "/data/ecomm/ajiob2b/processed/20230511/metadata/BrickDetails"
val B2B_PROCESSED_BRICK_DETAILS_PATH: String = "/data/ecomm/ajiob2b/processed/20230511/metadata/processedBrickDetailsPLP"
val B2B_IMAGE_PER_STYLE_CODE_PATH: String = "/data/ecomm/ajiob2b/processed/20230512/ImagePerStyleCode"
val LIVE_STYLES_DATE_WISE_PATH = "/data/ecomm/ajiob2b/processed/LiveStyleCodesDateWise"
val LIVE_STYLES_DATE_WISE_PATH_V2 = "/data/ecomm/ajiob2b/processed/LiveStyleCodesDateWiseV2"
val COMBINED_USER_PRODUCT_INTERACTIONS_PATH = "/data/ecomm/ajiob2b/processed/20230511/CombinedUPI"
val COMBINED_PACK_ATTRIBUTES_PATH = "/data/ecomm/ajiob2b/processed/20230511/CombinedPackAttributes"
val REFINED_STYLE_ATTRIBUTES_PATH = "/data/ecomm/ajiob2b/processed/20230512/RefinedStyleAttributes"
val GA_INTERACTIONS_PATH = "/data/ecomm/ajiob2b/processed/ga/ULevelProductGA"
val USER_CLUSTER_MAPPING_PATH = "/data/ecomm/ajiob2b/processed/20230511/AssignedUsersClusters.json"
val WISHLIST_PATH = "/data/ecomm/ajiob2b/processed/20230511/WishlistMaster"

In [ ]:
var upi = (
    ParquetToDF.getDF(COMBINED_USER_PRODUCT_INTERACTIONS_PATH)
    .filter(col(DATE).between("2023-04-07", "2023-05-06"))
    .filter(col(INTERACTIONTYPE) === "Order" && col("versionId").isNull)
    .withColumn("rank", dense_rank().over(Window.partitionBy(INTERACTION_CODE, "styleproduct").orderBy(DATE)))
    .filter(col("rank") === 1)
    .drop("rank")
)

var ga = (
    ParquetToDF.getDF(GA_INTERACTIONS_PATH)
      .withColumn(DATE, date_format(to_timestamp(col(DATE).cast(StringType), "yyyyMMdd"), "yyyy-MM-dd"))
      .filter(col(DATE).between("2023-04-07", "2023-05-06"))
)

var liveStyles = (
    ParquetToDF.getDF(LIVE_STYLES_DATE_WISE_PATH_V2)
    .withColumn(DATE, date_format(to_timestamp(col(DATE).cast(StringType), "yyyyMMdd"), "yyyy-MM-dd"))
)

var liveStylesData = (
    ParquetToDF.getDF("/data/ecomm/ajiob2b/processed/plp/liveStyles")
)

### Train/test individual checks

In [ ]:
var df = ParquetToDF.getDF("/data/ecomm/ajiob2b/processed/plp/features/individual/GARelated/dataset=train/Algorithm=GlobalPLP")
df.printSchema

In [ ]:
df.groupBy(length(col("stylecode"))).count().show()

In [ ]:
var df = ParquetToDF.getDF("/data/ecomm/ajiob2b/processed/plp/features/individual/UPIRelated/dataset=train/Algorithm=GlobalPLP")
df.printSchema

In [ ]:
df.groupBy(DAYS).agg(countDistinct(STYLECODE) as "count", sum("revenue") as "revenue", sum("quantity") as "quantity").orderBy(DAYS).show(false)

In [ ]:
upi.withColumnRenamed("styleproduct", STYLECODE).join(liveStyles.join(liveStyles.filter(col(DATE) === "2023-05-09").select(STYLECODE), Seq(STYLECODE)), Seq(STYLECODE, DATE)).groupBy(DATE).agg(countDistinct(STYLECODE) as "count", sum("orderprice") as "revenue", sum("orderquantity") as "quantity").orderBy(DATE).show(30, false)

### yData Individual checks

In [ ]:
var df = ParquetToDF.getDF("/data/ecomm/ajiob2b/processed/plp/features/individual/UPIRelated/dataset=yData/Algorithm=GlobalPLP")
df.printSchema

In [ ]:
df.groupBy(DAYS).agg(countDistinct(STYLECODE) as "count", sum("revenue") as "revenue", sum("quantity") as "quantity").orderBy(DAYS).show(false)

In [ ]:
upi.withColumnRenamed("styleproduct", STYLECODE).join(liveStyles.filter(col(DATE) === "2023-05-09"), Seq(STYLECODE, DATE)).agg(countDistinct(STYLECODE) as "count", sum("orderprice") as "revenue", sum("orderquantity") as "quantity").show

### Xdata sanity checks

In [ ]:
var features = ParquetToDF.getDF("/data/ecomm/ajiob2b/processed/plp/features/XData/dataset=train/Algorithm=GlobalPLP")
features.printSchema

In [ ]:
var df = features.select(STYLECODE, "1_RevenuePerView", "1_Consideration", "1_QuantityPerView", "1_WishlistPerView", "1_revenue", "1_quantity", "1_productViews")
df.printSchema

In [ ]:
df.filter(col("1_RevenuePerView") =!= 0.0).show(5, false)

In [ ]:
features.select("1_revenue", "1_quantity", "1_productViews").describe().show(false)

In [ ]:
(33092.64/339110.92000000004)/(2.0 - (302.0/3076.0))

In [ ]:
upi.filter(col(DATE) === "2023-04-28" && col("styleproduct") === "420392462").agg(sum("orderprice")).show

### YData sanity checks

In [ ]:
var y = ParquetToDF.getDF("/data/ecomm/ajiob2b/processed/plp/features/YData/Algorithm=GlobalPLP/clusterid=global")
y.printSchema

In [ ]:
y.describe().show(false)

In [ ]:
upi.filter(col(DATE) === "2023-04-30").groupBy("styleproduct").agg(sum("orderprice") as "revenue", sum("orderquantity") as QUANTITY).show(5, false)

In [ ]:
ga.filter(col(DATE) === "2023-04-30").groupBy(STYLECODE).agg(sum(PRODUCT_VIEWS) as PRODUCT_VIEWS).filter(col(STYLECODE).isin(Array("420392467", "464589772", "465586796"): _*)).show

In [ ]:
ga.filter(col(DATE) === "2023-04-30").groupBy(STYLECODE).agg(sum(PRODUCT_VIEWS) as PRODUCT_VIEWS).agg(max("productViews")).show

In [ ]:
upi.filter(col(DATE) === "2023-04-30").groupBy("styleproduct").agg(sum("orderprice") as "revenue", sum("orderquantity") as QUANTITY).agg(max("revenue"), max(QUANTITY)).show(5, false)

In [ ]:
y.filter(col(STYLECODE).isin(Array("420392467", "464589772", "465586796"): _*)).show

In [ ]:
val nv = 237/2603.0
val nr = 66405.73000000001/415225.44000000006
val nq = 77/240.0
print(nr/(2-nv), nq/(2-nv))

### Combined rankings checks

In [ ]:
var rankings = ParquetToDF.getDF("/data/ecomm/ajiob2b/processed/plp/rankings")
var weights = ParquetToDF.getDF("/data/ecomm/ajiob2b/processed/plp/weights")
var combinedRanking = ParquetToDF.getDF("/data/ecomm/ajiob2b/processed/plp/combinedRanking")

In [ ]:
rankings.printSchema

In [ ]:
combinedRanking.printSchema

In [ ]:
combinedRanking.filter(col(PRODUCTRANK)<=5).orderBy(ALGORITHM, CLUSTERID).show(false)

In [ ]:
rankings.filter(col("Algorithm") === "GlobalPLP").filter(col(STYLECODE) === "465645536").show(5, false)

In [ ]:
rankings.join(weights, Seq("target", "Algorithm")).count

In [ ]:
var XData = ParquetToDF.getDF("/data/ecomm/ajiob2b/processed/plp/standardizedDataset/test")
XData.printSchema

In [ ]:
var targets = Array("RevenuePerView", "QuantityPerView")
targets.map(target => {
      if (!target.startsWith("yData")) {
        s"yData$target"
      } else {
        target
      }
    })

In [ ]:
var df = ParquetToDF.getDF("/data/ecomm/ajiob2b/processed/plp/combinedRanking/model=xgb/Algorithm=SegmentPLP")
df.printSchema

### Target data analysis

In [ ]:
var df = ParquetToDF.getDF("/data/ecomm/ajiob2b/processed/plp/features/YData")
df.printSchema

In [ ]:
df.filter(col("yDataRevenuePerView")>0.5).filter(col(CLUSTERID) === "global").show(5, false)

In [ ]:
upi.filter(col("styleproduct") === "420441999").groupBy(DATE).agg(sum("orderprice") as "revenue").orderBy(desc(DATE)).show(false)
ga.filter(col(STYLECODE) ==="420441999").groupBy(DATE).agg(sum("productViews") as "views").orderBy(desc(DATE)).show(false)

### Brick details Transformation

In [ ]:
var brickDetails = spark.read.parquet("/data/ecomm/ajiob2b/processed/20230511/metadata/BrickDetails")
brickDetails = brickDetails.withColumn("L1_CODE", substring(col("brickid"), 0, 4)).withColumn("L2_CODE", substring(col("brickid"), 0, 6))
brickDetails = brickDetails.as("bd1").join(
    brickDetails.select(col("brickname").as("L1_NAME"), col("brickid")).as("bd2"), col("bd2.brickid") === col("bd1.L1_CODE")
).join(
    brickDetails.select(col("brickname").as("L2_NAME"), col("brickid")).as("bd3"), col("bd3.brickid") === col("bd1.L2_CODE")
).filter(length(col("bd1.brickid")) > 6).select("bd1.brickid", "L1_NAME", "L2_NAME", "brickname").dropDuplicates().toDF("brickid", "l1name", "l2name", "brickname")

brickDetails.show(5, false)
DFToParquet.putDF("/data/ecomm/ajiob2b/processed/20230511/metadata/processedBrickDetailsPLP", brickDetails)
// Self join to same df to get the corresponding L1 and L2 names
// Input: processed/metadata/BrickDetails, output: processed/metadata/processedBrickDetailsPLP

In [ ]:
var brickDetails = ParquetToDF.getDF("/data/ecomm/ajiob2b/processed/20230511/metadata/processedBrickDetailsPLP")
brickDetails.printSchema

### Miscellaneous

In [ ]:
var df = ParquetToDF.getDF(COMBINED_PACK_ATTRIBUTES_PATH)
df.printSchema

In [ ]:
df.show(5)

In [ ]:
df.columns.foreach(println(_))

In [ ]:
upi.printSchema

In [ ]:
upi.show(1, false)

In [ ]:
upi.filter(col(DATE) === "2023-05-06").groupBy("styleproduct").agg(sum("orderprice"), sum("packprice"), sum("orderquantity"), sum("packquantity")).show(false)

In [ ]:
upi.filter(col("interactionCode") === "49408387" && col(DATE) === "2023-05-06").show(33, false)

In [ ]:
upi.filter(col("interactionCode") === "49408387").agg(sum("packprice"), sum("packquantity")).show(false)

In [ ]:
var upi_full = spark.read.parquet(COMBINED_USER_PRODUCT_INTERACTIONS_PATH)

In [ ]:
upi_full.filter($"interactionCode"==="49408387").select("date").distinct.show(false)

In [ ]:
upi_full.orderBy(desc("date")).show(1, false)

In [ ]:
upi_full.filter($"interactionCode"==="49408387"&&$"packquantity"=!=0).distinct.count

In [ ]:
upi.filter(col("interactionCode") === "49408387").agg(sum("packquantity")).show(false)

In [ ]:
upi.filter(col("interactionCode") === "49408387").select("interactiontype").distinct.show(false)

In [ ]:
upi_full.filter(col("interactionCode") === "49408387").distinct.show(false)

In [ ]:
upi_full.filter(col("interactionCode") === "49408387").groupBy("styleproduct").count.orderBy(desc("count")).show(5, false)

In [ ]:
upi_full.filter(col("interactionCode")==="420377466" && col("styleproduct")==="420377466").show(5, false)

In [ ]:
upi_full.filter(col("interactionCode") === "420377466" && col("packquantity") === 0).select("itemid").distinct.
except(upi_full.filter(col("interactionCode") === "420377466" && col("packquantity") === 1).select("itemid").distinct).show

In [ ]:
var refineStyleAttributes = spark.read.parquet("/data/ecomm/ajiob2b/processed/20230512/RefinedStyleAttributes")
refineStyleAttributes.printSchema

In [ ]:
refineStyleAttributes.groupBy(STYLECODE).agg(max("stylename"), max("brandname")).show(5, false)

In [ ]:
var df = ParquetToDF.getDF("/data/ecomm/ajiob2b/processed/plp/output/uc/RankingWithStyleAttributes")
df.printSchema

In [ ]:
df.groupBy(CLUSTERID).agg(countDistinct(STYLECODE) as "count", count(STYLECODE) as "count2").show()

In [ ]:
var df2 = ParquetToDF.getDF("/data/ecomm/ajiob2b/processed/plp/combinedRanking/model=xgb")
df2.printSchema

In [ ]:
df2.groupBy(CLUSTERID).count().show

In [ ]:
var images = ParquetToDF.getDF(B2B_IMAGE_PER_STYLE_CODE_PATH)
images.printSchema

In [ ]:
ga.filter(col(STYLECODE) === "466037126").groupBy(DATE).agg(sum(PRODUCT_VIEWS) as PRODUCT_VIEWS, sum(PRODUCT_CLICKS) as PRODUCT_CLICKS).orderBy(desc(DATE)).show(false)

In [ ]:
var brickDetails = ParquetToDF.getDF("/data/ecomm/ajiob2b/processed/20230511/metadata/processedBrickDetailsPLP")
brickDetails.printSchema

In [ ]:
brickDetails.filter(col("l2name") === "830502").show()

In [ ]:
val df = ParquetToDF.getDF("/data/ecomm/ajiob2b/processed/20230511/metadata/BrickDetails")
df.printSchema

In [ ]:
df.filter(col("brickid").startsWith("830311001")).show(false)

In [ ]:
var df1 = ParquetToDF.getDF("/data/ecomm/ajiob2b/processed/plp/combinedRanking/model=lr/Algorithm=GlobalPLP")
var df2 = ParquetToDF.getDF("/data/ecomm/ajiob2b/processed/plp/combinedRanking/model=xgb/Algorithm=GlobalPLP")
df1.printSchema

In [ ]:
df1.orderBy(PRODUCTRANK).show(5, false)
df2.orderBy(PRODUCTRANK).show(5, false)